In [1]:
    import ibis.omniscidb, dask, intake, sqlalchemy, pandas, pyarrow as arrow, altair, h5py as hdf5
    

<!-- END_TEASER -->

<!--    

In [2]:
    
    #import refs
    #from refs import __doc__ as refs
    over_time = pandas.read_csv('annual_tags.csv').set_index('year').fillna('')
    __import__('warnings').simplefilter("ignore")
    import IPython
   

 
-->

# `ibis` as a generalized query tool for different backends.

In [our most recent `ibis` post] we look at querying & retrieving data using a familiar `pandas`-like interface.
This example focused on the fluent API that `ibis` provides to query structure from a SQLite database, a single backend.
In this post, we'll explore `ibis`'s ability to answer questions about data using to different `ibis` backends.

## `ibis` in the scientific Python ecosystem.

Before we continue into the technical nitty-gritty of `ibis`, we'll consider `ibis` in the greater historical context of the scientific Python ecosystem.

The design of quite a few high-level tools in the scientific Python world can be tracked back to the holistic `blaze` ecosystem that offered ways _**store, describe, query, and process**_ data.
`blaze` had ambitious goals and is now defunct, but its influences resonate throughout
the scientific python community development in the success of projects the focus specific features of a data ecosystem like:

* `dask` processing data.
* `intake` for describing data.
* `ibis` for querying data.

Throughout the rest of this document we'll highlight the ability of `ibis` to generically prescribe
query expressions across different data storage systems.

### The design of [`ibis` backends][backends].

Currently, `ibis` supports __>10__ backends.
    
    >>> dir(ibis)
    [...HDFS...WebHDFS...bigquery...clickhouse...hdf5...impala...omniscidb...pandas...pyspark...spark...sql...sqlite...]
    
A backend takes an `ibis` query expression and applies computation, _and the query is independent of the computation_.
A backend implementation, that can be queried with `ibis`, has one of the three following architectures.

1. Direct execution backends - `pandas and hdf5`. 
2. Expression generating backends that create `sqlalchemy` expressions - `ibis.sql`.
3. String generating backends - `ibis.bigquery and ibis.omniscidb`

In the next few sections we'll unravel some of the different capiabilities of each approach.

In [3]:
display(IPython.display.Markdown(F"""## A data-driven history of `ibis` compatability.

The table below looks at over 2000 issues in the ibis project.
It provides an annual looked at the issues tagged in `ibis`
for different backends over __{len(over_time)}__ years.
"""), over_time)

## A data-driven history of `ibis` compatability.

The table below looks at over 2000 issues in the ibis project.
It provides an annual looked at the issues tagged in `ibis`
for different backends over __6__ years.


,omnisci,spark,postgres,bigquery,pandas,sqlite,impala,kudu,geospatial,clickhouse,mysql,sqlalchemy
year,,,,,,,,,,,,
2015,,,2,,2,25,52,,,,,17
2016,,,3,,,2,4,,,,,3
2017,,1,21,15,49,10,15,,,8,,10
2018,31,,10,71,35,8,17,,,9,2,2
2019,33,22,17,12,32,1,4,,7,1,2,5
2020,38,3,4,2,4,1,2,1,3,4,4,


> We note an early focus `ibis.sqlite, sqlalchemy and ibis.impala`. 
Later, work began on the `pandas` backend rounding out the three different types of backgrounds.
From this point, improvements were made to these key backends as `ibis.clickhouse, ibis.spark and "postgres"`. Recently, 
For the past 3 years, Quansight, in partnership with OmniSci, added the `ibis.omniscidb`
string generating backend. Since Quansight Labs has taken on a role as a community maintainer 
for `ibis`. This collaboration introduced geospatial functionality to `ibis`, and we 
have on going efforts to introduce [SQL Server][sql-server] support

### Currently, there is an ongoing effort to add sqlserver backends. 

Leave a comment the backends what you like to see? `dask`? `altair`?

## `ibis` direct execution.

`ibis` direct execution backends like `pandas and hdf5` operate on conventional in-memory python objects.
`pandas` is the gold standard for structured data in python, and inspires the api for `ibis`.

In [4]:
    pd = ibis.pandas.connect({'A': pandas.util.testing.makeDataFrame()})

`pd` is an `ibis` backend based off `pandas.DataFrame` objects.

In [5]:
    expression = pd.table('A').head()

`expression` is an `ibis` query, that has `expression.compile` and `expression.execute` methods.
We'll recognize the __execute__ method when we return `pandas.DataFrame`s from `ibis` expression.
The __compile__ method does not trigger any computation, rather it constructs an intermediate form
that is interpretted by a backend.

In [6]:
    >>> assert isinstance(expression.compile(), ibis.expr.types.TableExpr)

In the case of direction execution backends, the `expression` compiles to an the original `ibis` 
expression.  And the computation is carried out based on a set of recipes defined in `ibis`.

In general, we would typically do this work directly in `pandas`, however this work is
practical in mocking tests for expressions independent of backends.

> Learn more about the [HDF5 direct execution backend in the `ibis` tests][test-hdf5].

## `ibis` expression generating backends.

In [7]:
    db = ibis.sqlite.connect('lahmansbaseballdb.sqlite')
    expression = db.table('halloffame').head()

Expression generating backends operate on [SQL] databases that interoperator with `sqlalchemy`.

In [8]:
    >>> assert isinstance(expression.compile(), sqlalchemy.sql.Select)

    
In the case of expression generating backends, the intermediate representation is a `sqlalchemy` object.
`sqlalchemy` is _The Database Toolkit for Python_, and `ibis` leverages it compatability
with traditional [SQL] databases.
    


## `ibis` string generating backends.

```bash
pip install --upgrade ibis-framework[omniscidb]
```

String generating backends allow `ibis` to interface with big data systems that manage 
their own computation. For example, we may connect to an example `omnisci` database.
    

In [9]:
    import ibis.omniscidb

In [10]:
    omnisci = ibis.omniscidb.connect(host='metis.omnisci.com', user='demouser', password='HyperInteractive', port=443, database='omnisci', protocol='https')

    
`omnisci` is described as a string generating backend because the intermediate representation of the
query is a flavor of SQL.

In [11]:
    expression = omnisci.table('upstream_reservoir').head()

    
A string generating expression compiles to `ibis.omniscidb` flavored [SQL], while `ibis.bigquery` may have a different string representatin.

In [12]:
    >>> expression.compile()

'SELECT *\nFROM upstream_reservoir\nLIMIT 5'

> Major credit goes to [@xmnlab] in his heroic PR to introduce `ibis.omniscidb` into `ibis`. You can watch
the drama play out in this [Github Issue][omnisci-pr]. If you'd like to learn more about [OmniSci] and
`ibis.omniscidb` checkout the following links.
> * [OmniSci][omnisci]
> * [Quansight Labs- Ibis: Python data analysis productivity framework][labs-post]

## Conclusion

We'd like to thank the maintainers of the `ibis` for
their and effort in supporting the `ibis` community.

<!--

In [15]:
!jupyter nbconvert --to markdown --TemplateExporter.exclude_input=True the-ibis-backends.ipynb

[NbConvertApp] Converting notebook the-ibis-backends.ipynb to markdown
[NbConvertApp] Writing 13723 bytes to the-ibis-backends.md


-->